In [0]:
from pyspark.sql.types import *

from pyspark.sql.functions import * 

from delta.tables import *

In [0]:
emp_src_data=[(1,'varun','cincinnati','usa',5139180740)]

emp_src_schema=StructType(
                             [
                              StructField('emp_id',IntegerType(),False),
                              StructField('emp_name',StringType(),False),
                              StructField('city',StringType(),False),
                              StructField('country',StringType(),False),
                              StructField('phno',LongType(),False)
                             ]
                         )

emp_df=spark.createDataFrame(emp_src_data,emp_src_schema)

display(emp_df)

emp_id,emp_name,city,country,phno
1,varun,cincinnati,usa,5139180740


In [0]:
%sql
create or replace table dim_dest_emp
(
        emp_id int,
        emp_name string,
        emp_city string,
        emp_cntry string,
        emp_contact long
)using delta 
 location "FileStore/tables/delta_merge" ;

select * from dim_dest_emp


emp_id,emp_name,emp_city,emp_cntry,emp_contact


In [0]:
emp_df.createOrReplaceTempView('src_emp_table')

#SQL 

In [0]:
%sql 

merge into dim_dest_emp as t
using src_emp_table as s
      on t.emp_id=s.emp_id 
      when matched 
                    then 
                         update set 
                                     t.emp_name=s.emp_name,
                                     t.emp_city=s.city,
                                     t.emp_cntry=s.country,
                                     t.emp_contact=s.phno
      when not matched 
                      then 
                           insert(emp_id,emp_name,emp_city,emp_cntry,emp_contact) values (s.emp_id,s.emp_name,s.city,s.country,s.phno)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql 

select * from dim_dest_emp

emp_id,emp_name,emp_city,emp_cntry,emp_contact
1,varun,cincinnati,usa,5139180740


senario:2

In [0]:

emp_data=[(1,'varun','mobile','usa',5139180740),(2,'sahithi','london','uk',56489456165)]

emp_df_2=spark.createDataFrame(emp_data,emp_src_schema) 

emp_df_2.show()

+------+--------+------+-------+-----------+
|emp_id|emp_name|  city|country|       phno|
+------+--------+------+-------+-----------+
|     1|   varun|mobile|    usa| 5139180740|
|     2| sahithi|london|     uk|56489456165|
+------+--------+------+-------+-----------+



In [0]:
emp_df_2.createOrReplaceTempView('src_table_data')

In [0]:
%sql 

select * from dim_dest_emp

emp_id,emp_name,emp_city,emp_cntry,emp_contact
1,varun,cincinnati,usa,5139180740


In [0]:
%sql 

merge into dim_dest_emp as t 
using src_table_data as s 
on t.emp_id=s.emp_id 
   when matched 
                then 
                      update set 
                                    t.emp_name=s.emp_name,      
                                    t.emp_city=s.city,              
                                    t.emp_cntry=s.country,
                                    t.emp_contact=s.phno 
   when not matched 
                    then 
                         insert(emp_id,emp_name,emp_city,emp_cntry,emp_contact) values (s.emp_id,s.emp_name,s.city,s.country,s.phno) ;


select * from dim_dest_emp;


emp_id,emp_name,emp_city,emp_cntry,emp_contact
2,sahithi,london,uk,56489456165
1,varun,mobile,usa,5139180740


#method_2:pypsark

In [0]:
data=[(2,'kusuma','london','uk',58361654),(3,'ganesh','hyderabad','india',9550102244)]

emp_df_3=spark.createDataFrame(data,emp_src_schema) 

emp_df_3.show()

+------+--------+---------+-------+----------+
|emp_id|emp_name|     city|country|      phno|
+------+--------+---------+-------+----------+
|     2|  kusuma|   london|     uk|  58361654|
|     3|  ganesh|hyderabad|  india|9550102244|
+------+--------+---------+-------+----------+



In [0]:
emp_table_instance=DeltaTable.forName(spark,"dim_dest_emp")


In [0]:

emp_table_instance.alias('t').merge(
                                          source=emp_df_3.alias('s'),
                                          condition="t.emp_id=s.emp_id"
                                   )\
                                   .whenMatchedUpdate(
                                                        set=
                                                             {
                                                                 "emp_name":"s.emp_name",
                                                                 "emp_city":"s.city",
                                                                 "emp_cntry":"s.country",
                                                                 "emp_contact":"s.phno"
                                                             }
                                                     )\
                                   .whenNotMatchedInsert(
                                                           values=
                                                           {
                                                               "emp_id":"s.emp_id",
                                                               "emp_name":"s.emp_name",
                                                               "emp_city":"s.city",
                                                               "emp_cntry":"s.country",
                                                               "emp_contact":"s.phno"
                                                           }
                                                        )\
                                   .execute()

In [0]:
%sql 

select * from dim_dest_emp

emp_id,emp_name,emp_city,emp_cntry,emp_contact
3,ganesh,hyderabad,india,9550102244
1,varun,mobile,usa,5139180740
2,kusuma,london,uk,58361654


In [0]:
display(emp_table_instance.history())

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2024-06-07T16:00:36.000+0000,8432478468657958,paruchuri7781@gmail.com,MERGE,"Map(predicate -> [""(emp_id#6420 = emp_id#6175)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2743043214707296),0607-140626-dbonvaq0,3,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 3261, numTargetBytesRemoved -> 1616, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 6329, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2552, numTargetRowsUpdated -> 1, numOutputRows -> 2, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 2, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2808)",null,Databricks-Runtime/12.2.x-scala2.12
3,2024-06-07T15:16:02.000+0000,8432478468657958,paruchuri7781@gmail.com,MERGE,"Map(predicate -> [""(emp_id#4811 = emp_id#4365)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2743043214707296),0607-140626-dbonvaq0,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 3226, numTargetBytesRemoved -> 1638, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 7277, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2839, numTargetRowsUpdated -> 1, numOutputRows -> 2, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 2, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 3499)",null,Databricks-Runtime/12.2.x-scala2.12
2,2024-06-07T15:03:39.000+0000,8432478468657958,paruchuri7781@gmail.com,MERGE,"Map(predicate -> [""(emp_id#3329 = emp_id#2)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2743043214707296),0607-140626-dbonvaq0,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 1638, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 10540, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2711, numTargetRowsUpdated -> 0, numOutputRows -> 1, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 6106)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-06-07T14:54:06.000+0000,8432478468657958,paruchuri7781@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(2743043214707296),0607-140626-dbonvaq0,0,WriteSerializable,false,Map(),null,Databricks-Runtime/12.2.x-scala2.12
0,2024-06-07T14:24:52.000+0000,8432478468657958,paruchuri7781@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(2743043214707296),0607-140626-dbonvaq0,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


#AUDIT LOG

In [0]:
%sql 

create table audit_log
(
  operation string,
  updated_time timestamp,
  user_name string,
  notebook_name string,
  numTargetRowsUpdated int,
  numTargetRowsInserted int,
  numTargetRowsDeleted int
)

In [0]:
last_operation_df=emp_table_instance.history(1) 
display(last_operation_df)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2024-06-07T16:00:36.000+0000,8432478468657958,paruchuri7781@gmail.com,MERGE,"Map(predicate -> [""(emp_id#6420 = emp_id#6175)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2743043214707296),0607-140626-dbonvaq0,3,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 3261, numTargetBytesRemoved -> 1616, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 6329, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2552, numTargetRowsUpdated -> 1, numOutputRows -> 2, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 2, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2808)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
last_operation_df.select(explode('operationMetrics')).show()

+--------------------+-----+
|                 key|value|
+--------------------+-----+
| numTargetRowsCopied|    0|
|numTargetRowsDeleted|    0|
| numTargetFilesAdded|    2|
| numTargetBytesAdded| 3261|
|numTargetBytesRem...| 1616|
|numTargetDeletion...|    0|
|numTargetRowsMatc...|    1|
|     executionTimeMs| 6329|
|numTargetRowsInse...|    1|
|numTargetRowsMatc...|    0|
|          scanTimeMs| 2552|
|numTargetRowsUpdated|    1|
|       numOutputRows|    2|
|numTargetDeletion...|    0|
|numTargetRowsNotM...|    0|
|numTargetChangeFi...|    0|
|       numSourceRows|    2|
|numTargetFilesRem...|    1|
|numTargetRowsNotM...|    0|
|       rewriteTimeMs| 2808|
+--------------------+-----+



In [0]:
last_operation_df.createOrReplaceTempView('history_view')





operationMetrics
"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 3261, numTargetBytesRemoved -> 1616, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 6329, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2552, numTargetRowsUpdated -> 1, numOutputRows -> 2, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 2, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2808)"


In [0]:
%sql
select operationMetrics from history_view

operationMetrics
"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 3261, numTargetBytesRemoved -> 1616, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 6329, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2552, numTargetRowsUpdated -> 1, numOutputRows -> 2, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 2, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2808)"


In [0]:
last_operation_df.select(col('operationMetrics.numTargetRowsDeleted'),col('operationMetrics.numTargetRowsInserted')).show()

+--------------------+---------------------+
|numTargetRowsDeleted|numTargetRowsInserted|
+--------------------+---------------------+
|                   0|                    1|
+--------------------+---------------------+



In [0]:
explode_df=last_operation_df.select("operation","timestamp","userName","notebook.notebookId",explode(last_operation_df.operationMetrics).alias('metric_name','metric_value'))
filtered_exploded_df=explode_df.filter(col('metric_name').isin(['numTargetRowsDeleted','numTargetRowsUpdated','numTargetRowsInserted']))
filtered_exploded_df.show() 

+---------+-------------------+--------------------+----------------+--------------------+------------+
|operation|          timestamp|            userName|      notebookId|         metric_name|metric_value|
+---------+-------------------+--------------------+----------------+--------------------+------------+
|    MERGE|2024-06-07 16:00:36|paruchuri7781@gma...|2743043214707296|numTargetRowsDeleted|           0|
|    MERGE|2024-06-07 16:00:36|paruchuri7781@gma...|2743043214707296|numTargetRowsInse...|           1|
|    MERGE|2024-06-07 16:00:36|paruchuri7781@gma...|2743043214707296|numTargetRowsUpdated|           1|
+---------+-------------------+--------------------+----------------+--------------------+------------+



In [0]:
final_df=filtered_exploded_df.groupBy('operation','timestamp','userName','notebookId')\
                    .pivot('metric_name')\
                    .agg(min('metric_value')) 

final_df.show()

+---------+-------------------+--------------------+----------------+--------------------+---------------------+--------------------+
|operation|          timestamp|            userName|      notebookId|numTargetRowsDeleted|numTargetRowsInserted|numTargetRowsUpdated|
+---------+-------------------+--------------------+----------------+--------------------+---------------------+--------------------+
|    MERGE|2024-06-07 16:00:36|paruchuri7781@gma...|2743043214707296|                   0|                    1|                   1|
+---------+-------------------+--------------------+----------------+--------------------+---------------------+--------------------+



In [0]:
final_df.createOrReplaceTempView('final_insert_data')

In [0]:
%sql 

insert into audit_log 
select * from final_insert_data 

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql 

select * from audit_log 

operation,updated_time,user_name,notebook_name,numTargetRowsUpdated,numTargetRowsInserted,numTargetRowsDeleted
MERGE,2024-06-07T16:00:36.000+0000,paruchuri7781@gmail.com,2743043214707296,0,1,1
